In [2]:
!pip install torch==2.2.1
!pip install sentence-transformers==2.6.0
!pip install llama-index==0.10.20
!pip install python-dotenv==1.0.1
!pip install llama-index-embeddings-huggingface==0.1.4
!pip install llama-index-readers-web==0.1.9
!pip install youtube_transcript_api==0.6.2
!pip install llama-index-readers-youtube-transcript==0.1.4

  Using cached torch-2.2.1-cp39-cp39-win_amd64.whl.metadata (26 kB)
Using cached torch-2.2.1-cp39-cp39-win_amd64.whl (198.5 MB)
  Attempting uninstall: torch
    Found existing installation: torch 2.3.0
    Uninstalling torch-2.3.0:
      Successfully uninstalled torch-2.3.0
  Using cached sentence_transformers-2.6.0-py3-none-any.whl.metadata (11 kB)
  Using cached scikit_learn-1.4.2-cp39-cp39-win_amd64.whl.metadata (11 kB)
  Using cached scipy-1.13.0-cp39-cp39-win_amd64.whl.metadata (60 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
Using cached sentence_transformers-2.6.0-py3-none-any.whl (163 kB)
Using cached scikit_learn-1.4.2-cp39-cp39-win_amd64.whl (10.6 MB)
Using cached scipy-1.13.0-cp39-cp39-win_amd64.whl (46.2 MB)
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)
  Using cached llama_index-0.10.20-py3-none-any.whl.metadata (8.8 kB)
  Using cached llama_index_agent_openai-0.1.7-py3-none-any.whl.metadata (644 bytes)
Using cached llama_index-0.10

## Setup API keys

In [3]:
import os
import openai
from dotenv import load_dotenv
from dotenv import load_dotenv, find_dotenv

load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')
assert openai.api_key, 'Please set OPENAI_API_KEY!'

## Some Utility Functions

In [12]:
from llama_index.core import (
    Document,
    SimpleDirectoryReader,
    VectorStoreIndex,
    DocumentSummaryIndex,
    SummaryIndex,
    StorageContext,
    load_index_from_storage,
    get_response_synthesizer
)
from llama_index.readers.youtube_transcript import YoutubeTranscriptReader
from llama_index.core.tools import FunctionTool, QueryEngineTool, ToolMetadata
from llama_index.llms.openai import OpenAI
from llama_index.core.node_parser import SentenceSplitter

def get_youtube_transcript_doc(yt_links:list)->Document:
    return YoutubeTranscriptReader().load_data(ytlinks=yt_links)

def build_vector_index(doc:Document, persist_dir=None)->VectorStoreIndex:
    if persist_dir:
        storage_context = StorageContext.from_defaults(persist_dir=persist_dir)
        index = VectorStoreIndex.load_from_storage(storage_context)
    else:
        index = VectorStoreIndex.from_documents(doc, show_progress=True)
    return index

def build_summary_index(doc:Document, persist_dir=None)->SummaryIndex:
    if persist_dir:
        storage_context = StorageContext.from_defaults(persist_dir=persist_dir)
        index = DocumentSummaryIndex.load_from_storage(storage_context)
    else:
        # from https://docs.llamaindex.ai/en/stable/examples/index_structs/doc_summary/DocSummary/
        index = DocumentSummaryIndex.from_documents(doc, show_progress=True)
        # LLM (gpt-3.5-turbo)
        chatgpt = OpenAI(temperature=0, model="gpt-3.5-turbo")
        splitter = SentenceSplitter(chunk_size=1024)

        # default mode of building the index
        response_synthesizer = get_response_synthesizer(
            response_mode="tree_summarize", use_async=True
        )
        doc_summary_index = DocumentSummaryIndex.from_documents(
            doc,
            llm=chatgpt,
            transformations=[splitter],
            response_synthesizer=response_synthesizer,
            show_progress=True,
        )
    return index

def get_query_engine(index, similarity_top=3):
    return index.as_query_engine(similarity_top_k=similarity_top)

def get_youtube_tool():
    return FunctionTool.from_defaults(fn=get_youtube_transcript_doc)


In [14]:
import nest_asyncio

nest_asyncio.apply()

## Create an example Vector Index from a YouTube Link

In [15]:
# Intro to LLMs, Andrej Karpathy
yt_links = ["https://www.youtube.com/watch?v=zjkBMFhNj_g"]

yt_doc = get_youtube_transcript_doc(yt_links)
yt_vector_index = build_vector_index(yt_doc)

yt_engine = get_query_engine(yt_vector_index)
# yt_tool = get_youtube_tool(transcript_doc)

query_engine_tools = [
    QueryEngineTool(
        query_engine=yt_engine,
        metadata=ToolMetadata(
            name="youtube",
            description=(
                "YouTube transcript of Andrej Karpathy's Introduction to LLMs. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/19 [00:00<?, ?it/s]

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Summarizing documents:   0%|          | 0/1 [00:00<?, ?it/s]

current doc id: zjkBMFhNj_g


Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Summarizing documents:   0%|          | 0/1 [00:00<?, ?it/s]

current doc id: zjkBMFhNj_g


Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

## Summarize Document

In [19]:
yt_doc = get_youtube_transcript_doc(yt_links)
doc_id = "youtube_doc"
yt_doc[0].doc_id = doc_id
yt_summary_index = build_summary_index(yt_doc)

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

<string>:3: RuntimeWarning: coroutine 'run_async_tasks.<locals>._gather' was never awaited
<string>:3: RuntimeWarning: coroutine 'Dispatcher.span.<locals>.async_wrapper' was never awaited


Summarizing documents:   0%|          | 0/1 [00:00<?, ?it/s]

current doc id: youtube_doc


Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Summarizing documents:   0%|          | 0/1 [00:00<?, ?it/s]

current doc id: youtube_doc


Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

In [20]:
print(yt_summary_index.get_document_summary(doc_id))

ValueError: doc_id zjkBMFhNj_g not in index

## Query the Index

In [6]:
query = "What's a neural network?"
response = yt_engine.query(query)
print(response)

The main takeaways are that large language models operate in two modes of thinking - system one for quick, instinctive responses and system two for rational, conscious decision-making. These models are evolving to incorporate tool use, enabling them to perform tasks like data analysis and graph generation. The training process involves pre-training on internet text for knowledge and fine-tuning on Q&A documents for alignment, resulting in assistant models capable of providing helpful responses.


## Create a ReAct Agent

In [7]:
from llama_index.llms.openai import OpenAI
from llama_index.core.agent import ReActAgent

llm = OpenAI(model="gpt-4")
agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
)

## Test Agent

In [8]:
hint = "(use the youtube tool)"
queries = [
    # "Summarize to just the main takeaways.",
    # "What is a LLM according to Andrej Karpathy?",
    # "How does it work?",
    "How do you train ChatGPT?",
    "What's system 1 vs. system 2 thinking?",
    "What were the two main stages of AlphaGo?",
    "What will an LLM OS be able to do in a few years?",
    "How do you jailbreak an LLM?",
    "How do you do prompt injection?",
]

for query in queries:
    print(f"Query: {query}")
    response = agent.chat(f"{query}\n{hint}")
    print("-------------------------------------------------------------")

Query: How do you train ChatGPT?
Thought: The user wants to know how ChatGPT is trained. I can use the YouTube tool to fetch the transcript of Andrej Karpathy's Introduction to LLMs, which might contain information on how ChatGPT is trained.
Action: youtube
Action Input: {'input': 'How is ChatGPT trained?'}
Observation: ChatGPT is trained by utilizing a process that involves human-machine collaboration. This process entails humans providing helpful, truthful, and harmless labeling instructions to guide the training of the model. Additionally, the training process involves leveraging the capabilities of language models to sample answers, allowing humans to select and combine parts of these answers to create the best responses. This collaborative approach helps in efficiently and accurately training ChatGPT.
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: ChatGPT is trained through a process of human-machine collaboration. Humans provid

## Interactive chat

In [9]:
while True:
    user_input = input()
    if not user_input:
        break
    print(f"User: {user_input}")
    response = agent.chat(user_input)
    print(f"Agent: {response}")